In [1]:
import os
import sys
import time
import json
import logging
import warnings
import traceback
import numpy as np
import pandas as pd

from fmlc.baseclasses import eFMU

In [2]:
import matplotlib.pyplot as plt

# get root
try:
    root = os.path.dirname(os.path.abspath(__file__))
except:
    root = os.getcwd()
    
# afc modules
sys.path.append(os.path.dirname(root))
from afc.radiance.configs import get_config
from afc.radiance.forecast import Forecast
from afc.utility.weather import read_tmy3

# doper modules
from doper import DOPER, get_solver, resample_variable_ts, standard_report
from doper.computetariff import compute_periods
from doper.data.tariff import get_e19_2020_tariff
from doper.models.basemodel import default_output_list


In [3]:
from afc.optModel import control_model
from afc.defaultConfig import default_parameter
from afc.defaultConfig import ft2_to_m2
from afc.utility.plotting import plot_standard1

In [4]:
import json

with open('current_config.json', 'r') as f:
    config = json.load(f)

In [5]:
config

{'system_id': 'Test-AAA',
 'location_state': 'CA',
 'location_city': 'Berkeley',
 'location_orientation': '180',
 'room_width': '10',
 'room_height': '11',
 'room_depth': '15',
 'occupant_1_location': '2',
 'occupant_1_direction': '1',
 'occupant_2_location': '1',
 'occupant_2_direction': '2',
 'occupant_3_location': '5',
 'occupant_3_direction': '1',
 'occupant_brightness': '100',
 'occupant_glare': '100',
 'window_width': '4.5',
 'window_height': '8.5',
 'window_sill': '0.5',
 'window_count': '2',
 'system_type': 'EC-Sage',
 'system_zones': '3',
 'system_light': 'FLU',
 'system_lux': '400',
 'system_cooling': 'el',
 'system_cooling_eff': '3.5',
 'system_heating': 'el',
 'system_heating_eff': '0.95',
 'location_latitude': '37.85',
 'location_longitude': '-122.24',
 'interface_status': 'Updated Configuration.'}

In [12]:
def read_config(config):
    
    # Take the default parameter as a basis
    parameter = default_parameter(precompute_radiance=False)
    
    
    # Update the parameters with information from the Json file
    
    # Parameter radiance
    # Location of the building
    parameter['radiance']['location']['longitude'] = float(config['location_longitude'])
    parameter['radiance']['location']['latitude'] = float(config['location_latitude'])
    parameter['radiance']['location']['orient'] = int(config['location_orientation'])
    # Dimensions of the building
    parameter['radiance']['dimensions']['width'] = float(config['room_width'])*0.3048 #Needs to be converted in m
    parameter['radiance']['dimensions']['depth'] = float(config['room_depth'])*0.3048
    parameter['radiance']['dimensions']['height'] = float(config['room_height'])*0.3048
    # Needed calculations
    window_area = ft2_to_m2((float(config['window_height'])-float(config['window_sill'])) * float(config['window_width']) * float(config['window_count']))#in m2
    facade_area = ft2_to_m2(float(config['room_width']) * float(config['room_height'])) #in m2
    wwr = window_area/facade_area
    # Suraface occupied by the windows
    parameter['radiance']['wwr'] = round(wwr, 1)
    
    # Parameter facade
    parameter['facade']['window_area'] = window_area #in m2
    
    # Parameter zone
    parameter['zone']['heating_efficiency'] = float(config['system_heating_eff'])
    parameter['zone']['cooling_efficiency'] = float(config['system_cooling_eff'])
    parameter['zone']['param']['type'] = config['system_type']
    parameter['zone']['lighting_capacity'] = config['system_lux']
    
    # Parameter occupant
    parameter['occupant']['glare_max'] = float(config['occupant_glare'])
    parameter['occupant']['wpi_min'] = float(config['occupant_brightness'])
    
    return parameter

In [13]:
default_param = default_parameter(precompute_radiance=False)
updated_param = read_config(config)

In [16]:
print(default_param['occupant']['glare_max'])
print(updated_param['occupant']['glare_max'])

0.4
100.0


In [17]:
print(default_param['facade']['window_area'])
print(updated_param['facade']['window_area'])

6.2709552
6.689018880000001


In [20]:
print(default_param['zone']['cooling_efficiency'])
print(updated_param['zone']['cooling_efficiency'])

0.2857142857142857
3.5
